First connect through SSH tunnel to tools-login.wmflabs.org server.

Replace the username and password (below) with those you find in the replica.my.cnf file in the host home

In [1]:
import pandas as pd
import numpy as np
import pymysql                                               # to get, do pip install PyMySQL
import json
from sklearn.feature_extraction.text import CountVectorizer  #from the NLP tutorial
import re                                                    #from NLP tutorial

In [2]:
#This connects to the Wiki database

username = 'u15068'                # get from replica.my.cnf
password = 'G5V2xrWbdFTSkyoU'      # get from replica.my.cnf
db=pymysql.connect(host='localhost',port=4711,user=username,passwd=password,db='enwiki_p')

In [3]:
#Get the epa id numbers from the dataset on github

epa_articles = pd.read_csv("070916_edit_protected_articles.csv")
print(epa_articles.shape, len(epa_articles))
epa_ids=epa_articles["page_id"]
epa_ids[300:305]

(7526, 20) 7526


300    11857
301    11867
302    11921
303    11955
304    11968
Name: page_id, dtype: int64

In [4]:
# FUNCTION to Quary the wiki database for caterogylinks a page_id at a time
# To use it you need to be connected to a database through PyMySQL

# The result is a dictionary in the format {"page_id" : ["link1", "link2", ...]}

def GetDictWikiLinks(database, art_ids):
    data={}
    for i in range(len(art_ids)):       #to get them all len(epa_articles)
        cur = database.cursor()               #this is how you use pymusql apparently
        if i %1000 ==0:                 #progress print statement
            print("Quarry article", art_ids[i])
    
        #########################_____SQL_____##############################
        # This is the SQL command to quarry the Wikipedia database         #
        # The syntax of a quarry is:                                       #
        # SELECT [columns] FROM [table] WHERE [conditions] LIMIT [optional]#
        
        quarry="""
        SELECT pl_from, pl_title
        FROM pagelinks
        WHERE pl_from = %s"""
        cur.execute(quarry, np.asscalar(art_ids[i]))
        links = np.array(cur.fetchall())
    
        #Some quarries might be empty 
        if links.size is 0:
            print("Empty categorylink %s", art_ids[i])
        else:   
            key=links[0,0].decode("utf-8")
            data[key]=[links[l,1].decode("utf-8") for l in range(len(links[:,1]))]
    
        #might take about 10 minuts (10k quarries, or 30k links)
    
    return data

In [5]:
# Get the dictionary from Wikipedia database

data=GetDictWikiLinks(db, epa_ids)

Quarry article 25
Quarry article 39988
Quarry article 326123
Quarry article 1331018
Empty categorylink %s 4485200
Quarry article 4553108
Quarry article 13253481
Quarry article 24572232
Quarry article 41698932


In [11]:
data["15580374"][0:10]

['1782',
 '1862',
 '1931',
 '1931_Constitution_of_Ethiopia',
 '1990',
 '1990_Luzon_earthquake',
 '1st_Cavalry_Division_Horse_Cavalry_Detachment',
 '2002_Atlantic_hurricane_season',
 '2008',
 '2008_Chinese_milk_scandal']

In [9]:
# Write data as a JSON to file

with open('pl_epa.txt', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
        # Examples of querries!
        
        # to get the category links
        # SELECT cl_from, cl_to
        # FROM categorylinks
        # WHERE cl_from = %s
        
        # To get the page-links
        # SELECT pl_from, pl_title
        # FROM pagelinks
        # WHERE pl_from = %s
        # AND pl_namespace = 0
        
        # to get stuff from different tables:
        # SELECT page_id, cl_to
        # FROM page inner join categorylinks on(page.page_id = categorylinks.cl_from)
        # WHERE page_id = %s
        # actually that could have been simplyfied, but it nicely shows how to merge tables with "inner join"

In [ ]:
#then I would do the same thing, but parsing the words from the categories separately
#Then the same thing but including all the links in the article from the other table

You can keep the really specific topics that occur above some threshold of pages (5%? 10%?), then split the topics into words, filter stop words, and the bag of words to get things that are common enough to model.

In [ ]:
#I should try and run my own random forest, but first
#I should get a sample of non edit-protected articles (what if they have other kinds of protection?)
#then mix them with my dataset, make sure they are labelled, split in training and validation sets 
#train a random forest algorithm
#To do that I will have to modify the query and I might want to branch off this notebook, so first I should push it on github 
#And this is a good time to re-practice that